In [1]:
import selenium
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

import requests
import time
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import xlrd
import os
from tqdm import tqdm

import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter

In [2]:
stock = 'CEMAS'

PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

driver.get("https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse="+ stock)  
print(driver.title)
time.sleep(3)

CEMAS - Çemaş Döküm Hisse Senedi | İş Yatırım


In [3]:
def get_data():
    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('tbody', id = 'tbodyMTablo')
    trs = table.find_all('tr')

    #quarter_list = []
    #for i in range(0,4):
    #    quarter_list.append(soup.find_all('th', class_ = 'form-group')[0].find_all('option')[i].text)
    
    quarter_list = []
    for i in range(4):
        quarter_list.append(soup.find_all('span', id = 'select2-ddlMaliTabloDonem'+str(i+1)+'-container')[0].text)
    
    #print('get_data içindeki q_list', quarter_list)

    val_list = []
    for i in range(1,len(trs)):
        val_list.append(trs[i].find_all('td'))


    col_list = []
    for i in range(0,len(val_list)):
        col_list.append(val_list[i][0].text)

    col_list[0]
    col_list.remove('Gelir Tablosu')
    try:
        col_list.remove('Dipnot')
    except:
        pass


    try:
        col_list.remove('Nakit Akım Tablosu')
    except:
        pass

    q1 = []    
    for i in range(0,len(val_list)):
        try:
            q1.append(val_list[i][1].text)
        except:
            i = i+1

    q1[0]

    q2 = []    
    for i in range(0,len(val_list)):
        try:
            q2.append(val_list[i][2].text)
        except:
            i = i+1

    q2[0]


    q3 = []    
    for i in range(0,len(val_list)):
        try:
            q3.append(val_list[i][3].text)
        except:
            i = i+1

    q3[0]

    q4= []    
    for i in range(0,len(val_list)):
        try:
            q4.append(val_list[i][4].text)
        except:
            i = i+1   

    q4[0]

    q1 = [w.replace('.', '') for w in q1]
    q1 = list(map(float, q1))

    q2 = [w.replace('.', '') for w in q2]
    q2 = list(map(float, q2))

    q3 = [w.replace('.', '') for w in q3]
    q3 = list(map(float, q3))

    q4 = [w.replace('.', '') for w in q4]
    q4 = list(map(float, q4))

    col_list.insert(0,'')
    q1.insert(0,'')
    q2.insert(0,'')
    q3.insert(0,'')
    q4.insert(0,'')

    col_list.insert(72,'')
    q1.insert(72,'')
    q2.insert(72,'')
    q3.insert(72,'')
    q4.insert(72,'')

    col_list.insert(115,'')
    q1.insert(115,'')
    q2.insert(115,'')
    q3.insert(115,'')
    q4.insert(115,'')


    df = pd.DataFrame(index = col_list)
    df['q1'] = q1
    df['q2'] = q2
    df['q3'] = q3
    df['q4'] = q4

    df.columns = quarter_list

    # gelir tablosu ve dipnot'tan sonra gelen kısmı eliyoruz. 
    #df = df.iloc[0:124,:]
    return df, quarter_list

In [4]:
def update_periods(driver, period_list):
    driver.find_element_by_id('select2-ddlMaliTabloDonem1-container').click()
    time.sleep(1)
    #print(period_list[0])
    driver.find_elements_by_xpath("//*[contains(@id, '" + period_list[0]+ "')]")[0].click()

    driver.find_element_by_id('select2-ddlMaliTabloDonem2-container').click()
    time.sleep(1)
    #print(period_list[1])
    driver.find_elements_by_xpath("//*[contains(@id, '" + period_list[1]+ "')]")[0].click()

    driver.find_element_by_id('select2-ddlMaliTabloDonem3-container').click()
    time.sleep(1)
    #print(period_list[2])
    driver.find_elements_by_xpath("//*[contains(@id, '" + period_list[2]+ "')]")[0].click()

    driver.find_element_by_id('select2-ddlMaliTabloDonem4-container').click()
    time.sleep(1)
    #print(period_list[3])
    driver.find_elements_by_xpath("//*[contains(@id, '" + period_list[3]+ "')]")[0].click()

In [5]:
driver.find_elements_by_class_name("cookie-policy-close")[0].click()

In [6]:
driver.find_elements_by_id("page-4")[0].click()

In [7]:
driver.find_element_by_id('select2-ddlMaliTabloDonem1-container').click()

In [8]:
html_text = driver.page_source
soup = BeautifulSoup(html_text, 'lxml')
table = soup.find('tbody', id = 'tbodyMTablo')
trs = table.find_all('tr')

uls = soup.find_all('ul', id = 'select2-ddlMaliTabloDonem1-results')

period_list = []
for ul in uls:
    for li in ul.findAll('li'):
        period_list.append(li.text)
        
period_list = [period for period in period_list if period >= '2017/0']   

driver.find_element_by_id('select2-ddlMaliTabloDonem1-container').click()

In [9]:
df_list = []

while len(period_list) >= 4:
    html_text = driver.page_source
    df1,q_list = get_data()    
        
    df_list.append(df1)
    
    period_list = [p for p in period_list if p not in q_list]
    
    if len(period_list) == 0:
        break
    
    if len(period_list) >= 8:
        update_periods(driver, period_list)
    if len(period_list) % 4 == 0:
        update_periods(driver, period_list)        
    else:
        addition = len(period_list)%4
        for i in range(4-addition):
            period_list.append(period_list[-1])
    
    
    time.sleep(2)
    print(len(period_list))

16
12
8
4


In [10]:
len(df_list)

5

In [11]:
len_list = list(map(len,df_list))
len_list

[150, 150, 150, 150, 142]

In [12]:
short_dfs = list(filter(lambda x: x <150, len_list))
short_dfs

[142]

In [13]:
if len(short_dfs) > 0:
    df_to_fix = df_list[len_list.index(short_dfs[0])]
    print(df_to_fix.shape)
    df_to_fix.head()
    
    df_to_fix_backup = df_to_fix.copy()
    df_to_fix = df_to_fix_backup.copy()
    df_to_fix2= df_to_fix.iloc[np.r_[0:72,73:142],:]
    df_to_fix = df_to_fix2.iloc[np.r_[0:114,115:141],:]
    
    list1 = list(df_to_fix.iloc[:,0])
    list1.insert(7,0)
    list1.insert(17,0)
    list1.insert(23,0)
    list1.insert(36,0)
    list1.insert(39,0)
    list1.insert(50,0)
    list1.insert(53,0)
    list1.insert(72,0)
    list1.insert(95,0)
    list1.insert(115,0)
    
    list2 = list(df_to_fix.iloc[:,1])
    list2.insert(7,0)
    list2.insert(17,0)
    list2.insert(23,0)
    list2.insert(36,0)
    list2.insert(39,0)
    list2.insert(50,0)
    list2.insert(53,0)
    list2.insert(72,0)
    list2.insert(95,0)
    list2.insert(115,0)
    
    list3 = list(df_to_fix.iloc[:,2])
    list3.insert(7,0)
    list3.insert(17,0)
    list3.insert(23,0)
    list3.insert(36,0)
    list3.insert(39,0)
    list3.insert(50,0)
    list3.insert(53,0)
    list3.insert(72,0)
    list3.insert(95,0)
    list3.insert(115,0)
    
    list4 = list(df_to_fix.iloc[:,3])
    list4.insert(7,0)
    list4.insert(17,0)
    list4.insert(23,0)
    list4.insert(36,0)
    list4.insert(39,0)
    list4.insert(50,0)
    list4.insert(53,0)
    list4.insert(72,0)
    list4.insert(95,0)
    list4.insert(115,0)
    
    df_fixed = pd.DataFrame(list(zip(list1,list2,list3,list4)))
    df_fixed.index = df_list[0].index
    df_fixed.columns = df_to_fix.columns
    
    df = pd.concat([df_list[0],df_list[1],df_list[2],df_list[3], df_fixed], axis = 1)

else:    
    df = pd.concat([df_list[0],df_list[1]], axis = 1)
    i = 2
    while i < len(df_list):
        df = pd.concat([df,df_list[i]], axis = 1)
        i = i+1

(142, 4)


In [14]:
df.index = df.index.str.strip()

In [15]:
cols_to_drop = len(set(df.columns)) - len(df.columns)
df = df.iloc[:,:cols_to_drop]
df = df.iloc[0:124,:]

In [16]:
os.chdir('C:\\Users\\Gökhan Akay\\Desktop\\BeautifulSoup\\Selenium Stocks')

In [17]:
# TAM için
df.to_excel(stock+'.xlsx')

In [21]:
df.transpose()

,,Dönen Varlıklar,Nakit ve Nakit Benzerleri,Finansal Yatırımlar,Ticari Alacaklar,Finans Sektörü Faaliyetlerinden Alacaklar,Diğer Alacaklar,Müşteri Sözleşmelerinden Doğan Varlıklar,Stoklar,Canlı Varlıklar,...,Sürdürülen Faaliyetlerden Seyreltilmiş Hisse Başına Kazanç,,Amortisman Giderleri,Kıdem Tazminatı,Finansman Giderleri,Yurtiçi Satışlar,Yurtdışı Satışlar,Net Yabancı Para Pozisyonu,Parasal net yabancı para varlık/(yükümlülük) pozisyonu,Net YPP (Hedge Dahil)
2021/9,,556935732.0,384777015.0,0.0,79988821.0,0.0,49669656.0,0.0,37908751.0,0.0,...,0.0,,9618932.0,2062337.0,-16393999.0,155724275.0,82785553.0,406221496.0,406221496.0,406221496.0
2021/6,,559914234.0,391545054.0,0.0,86635373.0,0.0,48975809.0,0.0,28975199.0,0.0,...,0.0,,6419739.0,1704337.0,-7515036.0,108656971.0,60450601.0,420099479.0,420099479.0,420099479.0
2021/3,,578952311.0,414947007.0,0.0,83044518.0,0.0,45861646.0,0.0,29928398.0,0.0,...,0.0,,3208873.0,1621205.0,-2605876.0,56217887.0,27324486.0,402775376.0,402775376.0,402775376.0
2020/12,,234727943.0,87346858.0,0.0,60952199.0,0.0,44524774.0,0.0,38313465.0,0.0,...,0.0,,12378599.0,1460498.0,-17640574.0,139427867.0,67428827.0,94722018.0,94722018.0,94722018.0
2020/9,,319908621.0,116969482.0,0.0,55154829.0,0.0,95428227.0,0.0,29570573.0,0.0,...,0.0,,9969349.0,1426961.0,-11436895.0,95917465.0,48188052.0,141620802.0,141620802.0,141620802.0
2020/6,,288243182.0,113917468.0,0.0,44608324.0,0.0,94321892.0,0.0,26422596.0,0.0,...,0.0,,6981838.0,1460498.0,-10419801.0,62310345.0,35924519.0,131746198.0,131746198.0,131746198.0
2020/3,,295050729.0,123911211.0,0.0,52625699.0,0.0,93826497.0,0.0,22380847.0,0.0,...,0.0,,3444200.0,1345763.0,-7404392.0,34986397.0,17625991.0,92453877.0,92453877.0,92453877.0
2019/12,,156384214.0,2102797.0,0.0,37333178.0,0.0,93828222.0,0.0,21481414.0,0.0,...,0.0,,8345072.0,1270373.0,-34952828.0,85957535.0,63123561.0,-71497063.0,-71497063.0,-71497063.0
2019/9,,161727659.0,3675276.0,0.0,36985554.0,0.0,96268019.0,0.0,22920012.0,0.0,...,0.0,,5732147.0,1170928.0,-28788572.0,65053873.0,53627171.0,-79334899.0,-79334899.0,-79334899.0
2019/6,,167135518.0,2327199.0,0.0,36982835.0,0.0,99337923.0,0.0,26555049.0,0.0,...,0.0,,3819238.0,1071254.0,-22932573.0,45148324.0,34613052.0,-81711262.0,-81711262.0,-81711262.0


In [ ]:
# getting prices.
import yfinance as yf
from datetime import date, datetime

today = date.today().strftime("%Y-%m-%d")
data = pd.DataFrame(yf.download(stock+'.IS','2017-01-01',today)['Close'])
result = data
result_index = list(result.index.strftime("%Y-%m-%d"))

index_list = ['2017-05-01','2017-08-01','2017-11-01','2018-02-01','2018-05-01','2018-08-01','2018-11-01','2019-02-01','2019-05-01',
              '2019-08-01','2019-11-01','2020-02-03','2020-05-04','2020-08-04','2020-11-02','2021-02-01','2021-05-03','2021-08-02','2021-11-01']

prices = list(result[result.index.isin(index_list)].sort_index(ascending = False).iloc[:,0])#.to_excel(stock + '_prices.xlsx')

In [ ]:
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter

tam_directory = 'C:\\Users\\Gökhan Akay\\Desktop\\Google Drive\\TAM_202112.xlsx'
examplary_stock = 'ASELS'
tam = load_workbook(tam_directory)

ws1 = tam.create_sheet(stock)
ws1.title = stock

# bilanço metrikleri
for row in range(130,199):
    for col in range(2,23):
        #char = chr(65+col)
        char = get_column_letter(col)
        worksheet = tam[examplary_stock]
        tam[stock][char + str(row)] = tam[examplary_stock][char + str(row)].value

# kolon isimleri (periyotlar)
for row in range(1,2):
    for col in range(3,23):
        char = get_column_letter(col)
        worksheet = tam[examplary_stock]
        tam[stock][char + str(row)] = tam[examplary_stock][char + str(row)].value     

# bilançodaki sayılar.
for row in range(3,126):
    for col in range(2,len(df.columns)+3):
        char = get_column_letter(col)
        tam[stock][char + str(row)] = df.reset_index().iloc[row-2,col-2]

# fiyat kolonu.      
for row in range(130,131):
    for col in range(3,3+len(prices)):
        char = get_column_letter(col)
        tam[stock][char + str(row)] = prices[col-3]        
        
tam.save(tam_directory)  